In [1]:
from STKBridge.Pipe import Pipeline

In [2]:
f = Pipeline()
# X = [6828.14, 96, 80, 9, 9, 0.5]
# sol = f.genSol(X)
# print("\r")
# print(sol)

Connection execution successfully
STK initialized.


In [3]:
# a,b,c =0.62, 0.25, 0.13
# val = a*(sol[0]/60) + b*(1-sol[1]/86400) + c*(1-sol[2]/150)
# print(val)

# Problem Data Relais

In [4]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class RelaisProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0)

    def _evaluate(self, x, out, *args, **kwargs):

        sol = f.genSol(x)

        out["F"] = np.array([-sol[0]], dtype=float)

# Sampling

In [5]:
from pymoo.core.sampling import Sampling

class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
         X = np.full((1, 6), None, dtype=object)
         
         X[0, 0] = np.random.randint(6428,6928) # a
         X[0, 1] = np.random.randint(80,120)    # i
         X[0, 2] = np.random.randint(60,100)    # RAAN
         X[0, 3] = np.random.randint(1,3)      # NumPlanes
         X[0, 4] = np.random.randint(1,4)      # NumSatxPlane
         # MAX 150 SATELLITES CONDITION:
         while(X[0, 3]*X[0, 4])>150:
              X[0, 3] = np.random.randint(1,30)      # NumPlanes
              X[0, 4] = np.random.randint(1,50)      # NumSatxPlane

         X[0, 5] = np.random.random(1)[0]*10
         
         return X

# Crossover

In [11]:
from pymoo.core.crossover import Crossover

class MyCrossover(Crossover):
    def __init__(self):

        # define the crossover: number of parents and number of offsprings
        super().__init__(2, 2)

    def _do(self, problem, X, **kwargs):

        # The input of has the following shape (n_parents, n_matings, n_var)
        _, n_matings, n_var = X.shape

        # The output owith the shape (n_offsprings, n_matings, n_var)
        # Because there the number of parents and offsprings are equal it keeps the shape of X
        Y = np.full_like(X, None, dtype=object)

        # for each mating provided
        for k in range(n_matings):

            # get the first and the second parent
            a, b = X[0, k, 0], X[1, k, 0]

            print(a, b)
            print(X)

            # prepare the offsprings
            off_a = np.zeros(6)
            off_b = np.zeros(6)

            for i in range(6):
                if np.random.random() < 0.5:
                    off_a[i] = a[i]
                    off_b[i] = b[i]
                else:
                    off_a[i] = b[i]
                    off_b[i] = a[i]

            # join the character list and set the output
            Y[0, k, 0], Y[1, k, 0] =list(off_a), list(off_b)

        return Y

# Mutation

In [8]:
from pymoo.core.mutation import Mutation

class MyMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):

        # for each individual

        # with a probabilty of 40% change semimajor axis
        r = np.random.random()
        if r < 0.4:
            rand = np.random.randint(-50,50)
            X[0, 0] += rand

        # with a probabilty of 40% change inclination
        r = np.random.random()
        if r < 0.4:
            rand = np.random.randint(-20,20)
            X[0, 1] += rand

        # with a probabilty of 40% change RAAN
        r = np.random.random()
        if r < 0.4:
            rand = np.random.randint(-20,20)
            X[0, 2] += rand


        # with a probabilty of 40% change NumPlanes
        r = np.random.random()
        if r < 0.4:
            rand = np.random.randint(1,5)
            X[0, 3] += rand

        # with a probabilty of 40% change NumSatsxPlane
        r = np.random.random()
        if r < 0.4:
            rand = np.random.randint(1,5)
            X[0, 4] += rand

        # with a probabilty of 40% change RAANincrement
        r = np.random.random()
        if r < 0.4:
            rand = np.random.random(1)[0]*5
            X[0, 5] += rand

       
        return X

In [9]:
from pymoo.core.duplicate import ElementwiseDuplicateElimination

class MyDuplicateElimination(ElementwiseDuplicateElimination):

    def is_equal(self, a, b):
        return a.X[0] == b.X[0]

In [12]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize


algorithm = NSGA2(pop_size=50,
                  sampling=MySampling(),
                  crossover=MyCrossover(),
                  mutation=MyMutation(),
                  eliminate_duplicates=MyDuplicateElimination())

res = minimize(RelaisProblem(),
               algorithm,
               ('n_gen', 100),
               seed=1,
               verbose=False)

Satellite Added!
Object: mysat fetched.
Walker Delta constellation generated.
Chain (Chain) object with sensors generated.
Object: Chain fetched.
Chain access computation for chain (Chain) executed.
Object: Sensors fetched.
Object: mysat fetched.
Object: mysat11 fetched.
Object: mysat12 fetched.
Object: mysat21 fetched.
Object: mysat22 fetched.
Object: Chain fetched.
Reset completed successfully.
6465 6465
[[[6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.0233257263183977]
  [6465 92 68 2 2 3.

TypeError: 'int' object is not subscriptable